In [2]:
import pandas as pd
!pip install fuzzywuzzy
from fuzzywuzzy import process


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip


/opt/homebrew/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
df = pd.read_csv('../../data/processed/productos-categorizados.csv')
df

,nombre,cantidad,enlace_foto,Imagen,Durabilidad
0,Fresón tarrina de 500 g.,500g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
1,Cogollos de lechuga bandeja de 3 uds.,3 por envase,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
2,Lechuga iceberg,400g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
3,DANONE Yogur natural elaborado con fermentos n...,960g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",medio
4,"Manojo de espárragos verdes, 250 g.",250g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
...,...,...,...,...,...
295,"ALPRO Bebida de almendras 100% vegetal, sin az...",1000ml,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",duradero
296,FONTER Agua mineral con gas botella de 1 l.,1000ml,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",duradero
297,"ALPRO Bebida de avena 100% vegetal, sin azúcar...",1000ml,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",duradero
298,Yogur natural azucarado 8 x 125 g.,1000g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",medio


In [10]:
def encontrar_mejor_coincidencia(texto, df, columna='nombre'):
  
    nombres_unicos = df[columna].unique()
    
    mejor_coincidencia, puntaje_similitud = process.extractOne(texto, nombres_unicos)
    
    return mejor_coincidencia, puntaje_similitud

# Texto que quieres comparar con los nombres en tu DataFrame
texto_a_comparar = 'lemon'

# Llamamos a la función y obtenemos la mejor coincidencia
mejor_coincidencia, puntaje_similitud = encontrar_mejor_coincidencia(texto_a_comparar, df)

print(f"La mejor coincidencia es: '{mejor_coincidencia}' con un puntaje de similitud de: {puntaje_similitud}")


La mejor coincidencia es: 'Limones 750 g.' con un puntaje de similitud de: 72


In [23]:
# palabras_clave = ['tomate ']

# # Función para filtrar las filas que contienen alguna de las palabras clave
# def filtrar_por_palabras_clave(df, palabras_clave):
#     # Esta función devuelve True si alguna de las palabras clave se encuentra en el texto
#     def contiene_palabra_clave(texto):
#         return any(palabra_clave in texto for palabra_clave in palabras_clave)
    
#     # Aplicamos la función a cada fila para cada columna que desees buscar
#     # Supongamos que queremos buscar en la columna 'nombre'
#     mask = df['nombre'].apply(contiene_palabra_clave)
    
#     # Filtramos el dataframe con la máscara
#     return df[mask]

# # Filtramos el DataFrame
# df_filtrado = filtrar_por_palabras_clave(df, palabras_clave)


In [38]:
textos_entrada = ['tomate ensalada', 'limones', 'CULTIVAMOS LO BUENO mandarinas', 'manzanas golden', 'plátano canario 1º']

# Columna del DataFrame en la que quieres buscar
columna_objetivo = 'nombre'

# Umbral de similitud para considerar una coincidencia adecuada
umbral_similitud = 90

def filtrar_filas_por_similitud_con_multiples_entradas(dataframe, textos, columna, umbral):
    # Función para determinar la similitud con cualquier texto de entrada y decidir si se incluye la fila
    def es_coincidencia_adecuada(texto_fila):
        for texto in textos:
            puntaje_similitud = process.extractOne(texto_fila, [texto])[1]
            if puntaje_similitud >= umbral:
                return True
        return False
    
    # Aplicamos la función a cada fila de la columna objetivo y filtramos el DataFrame
    mask = dataframe[columna].apply(es_coincidencia_adecuada)
    return dataframe[mask]

# Filtramos el DataFrame original para obtener solo las filas que cumplen con el criterio de similitud
df_filtrado = filtrar_filas_por_similitud_con_multiples_entradas(df, textos_entrada, columna_objetivo, umbral_similitud)
df_filtrado


,nombre,cantidad,enlace_foto,Imagen,Durabilidad
120,"CULTIVAMOS LO BUENO Mandarinas 1,5 kg.",1500g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
175,Tomate ensalada malla de 1 kg,1000g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
231,Manzanas golden a granel,1000g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
232,Limones 750 g.,750g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco
289,Plátano Canario 1ª a granel,1000g,https://www.compraonline.alcampo.es/images-v3/...,"<img src=""https://www.compraonline.alcampo.es/...",fresco


In [39]:
df_filtrado.to_csv('../../data/processed/frutitas.csv', index=False)